# Exercise 3

## Question 1
Load the prices data from SP500.csv to the SQLite database.

In [2]:
import sqlite3
import csv
from contextlib import closing

conn = sqlite3.connect("Exercise3.db")
cs = conn.cursor()
cs.execute("""
create table if not exists prices (
theday text primary key,
price real
);
""")
with closing(open('SP500.csv')) as datafile:
    reader = csv.DictReader(datafile, fieldnames=["date", "price"], delimiter='\t')
    for row in reader:
        cs.execute(f'insert into prices values ("{row["date"]}", {float(row["price"])})')
conn.commit()


## Question 2
Calibrate a GBM model to the prices of the 120 days prior to *2021-05-31*.

**Hint**:
1. Read the data from the database
2. Calibrate the GBM model in exactly the same way as in Question 3 of Exercise 2.

In [ ]:
import numpy as np
import sqlite3
import csv
from contextlib import closing

class GBM:
    def __init__(self, seed=None):
        self.mu = np.nan
        self.sigma = np.nan
        self.rng = np.random.default_rng()
        
    def calibrate(self, trajectory, Dt, num_bootstraps=100):
            """
            Calibrates the model's mu and sigma using bootstrapping and
            updates the object's state with the results.
            """       
            log_returns = np.diff(np.log(trajectory))
            
            bootstrap_means = []
            bootstrap_mean_squares = []
            
            for _ in range(num_bootstraps):
                bootstrap_sample = self.rng.choice(
                    log_returns, 
                    size=len(log_returns), 
                    replace=True
                )
                bootstrap_means.append(np.mean(bootstrap_sample))
                bootstrap_mean_squares.append(np.mean(bootstrap_sample**2))
            
            avg_mean = np.mean(bootstrap_means)
            avg_mean_square = np.mean(bootstrap_mean_squares)
            
            std_of_returns = np.sqrt(avg_mean_square - avg_mean**2)
            
            # Calculate the final parameter estimates
            est_sigma = std_of_returns / np.sqrt(Dt)
            est_mu = avg_mean / Dt + 0.5 * est_sigma**2
            
            # --- Key Step: Modify the object's state ---
            self.mu = est_mu
            self.sigma = est_sigma

Model parameters: -0.2212,   0.1419


In [13]:
test_date = '2021-05-31'
N = 120
model = GBM(seed=42)
prices = None
with closing(sqlite3.connect("Exercise3.db")) as conn:
    # Your code goes here
    cs = conn.cursor()
    cs.execute("SELECT price FROM prices WHERE theday < ? ORDER BY theday DESC LIMIT ?", (test_date, N))
    prices = [row[0] for row in cs.fetchall()]
    if len(prices) < N:
        raise ValueError("Not enough data points for the specified date and N.")
    model.calibrate(prices, Dt=1/252) 
print(F"Model parameters: % .4f,  % .4f" % (model.mu, model.sigma))


Model parameters: -0.1931,   0.1416


## Question 3
Use the calibrated GBM model to forecast the price on *2021-05-31*. What is the 95% confidence interval of the forecast?

**Hint**

Recall the formula for price forecast:
$$S_{t + \Delta t} = S_t \exp \left[
\left(
\mu - \frac{\sigma^2}{2}
\right)\Delta t + \sigma \left(
W_{t + \Delta t} - W_t
\right)
\right]
$$

Our price forecast is
$$
\mathbb E S_{t+\Delta t} = S_t \exp \left(
\mu \Delta t
\right)
$$

$S_t$ is the latest price we have, i.e. the price on *2021-05-30*. $\Delta t = 1/250$ - make sure you use the same $\Delta t$ in forecasting as you have done in calibration. $(\mu - \frac{\sigma^2}{2})t + \sigma(W_{t+\Delta t} - W_{t})$ has normal distribution with mean $(\mu - \frac{\sigma^2}{2})\Delta t$ and standard deviation $\sigma \sqrt{\Delta t}$. Use these parameters with scipy.stats.norm.ppf to calculate the 2.5% quantile and the 97.5% quantile of the distribution. Call them $q_1$ and $q_2$.

Then the required confidence interval is $(S_t e^{q_1}, S_t e^{q_2})$.

In [15]:
from scipy.stats import norm

# Your code goes here
prices_20210530 = np.array(prices)
St = prices_20210530[-1]

T = 1.0 / 252 # Time step in years

expected_price = St * np.exp(model.mu * T)
expected_price_std = St * model.sigma * np.sqrt(T)
print(f"Expected price on {test_date}: {expected_price:.2f} ± {expected_price_std:.2f}")
print(f"Expected price range: [{expected_price - expected_price_std:.2f}, {expected_price + expected_price_std:.2f}]")

Expected price on 2021-05-31: 3804.28 ± 33.97
Expected price range: [3770.31, 3838.25]
